In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import numpy as np
import tqdm
import random
import torch.nn.functional as F

# Cross-attention implementation

In [154]:
data_train = torch.tensor([np.random.choice([-1, 1], size=20) for _ in range(1000)])
data_train_dec = torch.tensor([np.random.choice([-1, 1], size=20) for _ in range(1000)])
data_test = torch.tensor([np.random.choice([-1, 1], size=20) for _ in range(400)])
data_test_dec = torch.tensor([np.random.choice([-1, 1], size=20) for _ in range(400)])

def one_hot_encoding(seq, vocab):
    if isinstance(seq, int):
        one_hot = torch.zeros(1, len(vocab), dtype=int)
        one_hot[:, vocab[seq]] = 1
    else:
        one_hot = torch.zeros((len(seq), len(vocab)), dtype=int)
        for i, spin in enumerate(seq):
            one_hot[i, vocab[spin]] = 1
    return one_hot

def mask_random_spin(sequence1, sequence2, vocab, mask_token=2):
    """
    Mask one random spin in a sequence of protein spins.
    
    Parameters:
    - sequence: a list or sequence of spins (integers)
    - mask_token: the token used to mask a spin (default is 2)
    
    Returns:
    - masked_sequence: a sequence similar to the input but with one spin masked
    - masked_position: the position of the spin that was masked
    """
    # Ensure the sequence can be converted to a list for masking
    sequence_list_1 = sequence1.numpy().tolist() if isinstance(sequence1, torch.Tensor) else list(sequence1)
    sequence_list_2 = sequence2.numpy().tolist() if isinstance(sequence2, torch.Tensor) else list(sequence1)
    # Choose a random position to mask, excluding the first spin
    mask_position = random.randint(1, len(sequence_list_1) - 1)
    
    # Mask the chosen position
    masked_sequence_1 = sequence_list_1.copy()
    masked_sequence_1[mask_position] = mask_token

    masked_sequence_2 = sequence_list_2.copy()
    masked_sequence_2[mask_position] = mask_token

    # Create an array of zeros with shape (len(sequence), len(vocab))
    one_hot_1 = one_hot_encoding(masked_sequence_1, vocab)
    one_hot_2 = one_hot_encoding(masked_sequence_2, vocab)
        
    return torch.tensor(one_hot_1, dtype=torch.float), torch.tensor(one_hot_2, dtype=torch.float), torch.tensor(mask_position)
    

In [155]:
class VanillaAttention(nn.Module):
    def __init__(self, embed_dim, a, max_seq_length, num_spins=3, dropout_rate=0.0):
        super(VanillaAttention, self).__init__()
        self.word_embeddings = nn.Linear(num_spins, embed_dim)
        self.position_embeddings = nn.Embedding(max_seq_length, embed_dim)
        self.a = a  # parameter controlling how important positions are
        self.value_weight = nn.Linear(embed_dim, embed_dim)
        self.query_weight = nn.Linear(embed_dim, embed_dim)
        self.key_weight = nn.Linear(embed_dim, embed_dim)
        self.fc = nn.Linear(embed_dim, embed_dim)  # output layer
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, s):
        #if not isinstance(s, torch.Tensor):
        #s = torch.tensor(s, dtype=torch.float)
        #masked_token = torch.tensor(masked_token, dtype=torch.float)

        position_ids = torch.arange(s.size(0), dtype=torch.long)
        #x = self.word_embeddings(s) + self.a*self.position_embeddings(position_ids)
        x = s + self.a*self.position_embeddings(position_ids)
        
        query = self.query_weight(x)
        key = self.key_weight(x)
        values = self.value_weight(x)
        
        # Simple attention score calculation (Dot product): this is equivalent to the interaction matrix
        scores = torch.matmul(query, key.transpose(-2, -1))  # Transpose last two dimensions for matrix multiplication
        scores = torch.softmax(scores, dim=-1)  # Apply softmax to scores to get probabilities

        # Apply attention scores to values
        attn_output = torch.matmul(scores, values)

        # Sum over the sequence length dimensions
        #attn_output = attn_output.sum(dim=1)
        output = self.fc(self.dropout(attn_output)) # should have size (20,3)

        return output
    
class CrossAttention(nn.Module):
    def __init__(self, embed_dim, a, max_seq_length, num_spins=3, dropout_rate=0.0):
        super(CrossAttention, self).__init__()
        self.word_embeddings = nn.Linear(num_spins, embed_dim)
        self.position_embeddings = nn.Embedding(max_seq_length, embed_dim)
        self.a = a  # parameter controlling how important positions are
        self.value_weight = nn.Linear(embed_dim, embed_dim)
        self.query_weight = nn.Linear(embed_dim, embed_dim)
        self.key_weight = nn.Linear(embed_dim, embed_dim)
        self.fc = nn.Linear(embed_dim, embed_dim)  # output layer
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, s, enc_output):
        #s = torch.tensor(s, dtype=torch.float)
        position_ids = torch.arange(s.size(0), dtype=torch.long)

        #x = self.word_embeddings(s) + self.a*self.position_embeddings(position_ids)
        x = s + self.a*self.position_embeddings(position_ids)

        query = self.query_weight(x)
        key = self.key_weight(enc_output)
        values = self.value_weight(enc_output)
        
        # Simple attention score calculation (Dot product): this is equivalent to the interaction matrix
        scores = torch.matmul(query, key.transpose(-2, -1))  # Transpose last two dimensions for matrix multiplication
        scores = torch.softmax(scores, dim=-1)  # Apply softmax to scores to get probabilities

        # Apply attention scores to values
        attn_output = torch.matmul(scores, values)

        # Sum over the sequence length dimensions
        #attn_output = attn_output.sum(dim=1)
        output = self.fc(self.dropout(attn_output))
        return output
    

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))


In [156]:
class EncoderLayer(nn.Module):
    def __init__(self, embed_dim, a, max_seq_length, num_spins, proj_layer_dim, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = VanillaAttention(embed_dim=embed_dim, a=a, max_seq_length=max_seq_length, num_spins=num_spins, dropout_rate=dropout)
        self.feed_forward = PositionWiseFeedForward(d_model=embed_dim, d_ff=proj_layer_dim)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        attn_output = self.self_attn(x)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x
    
class DecoderLayer(nn.Module):
    def __init__(self, embed_dim, a, max_seq_length, num_spins, proj_layer_dim, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = VanillaAttention(embed_dim=embed_dim, a=a, max_seq_length=max_seq_length, num_spins=num_spins, dropout_rate=dropout)# masking one of the word in the sequence
        self.cross_attn = CrossAttention(embed_dim=embed_dim, a=a, max_seq_length=max_seq_length, num_spins=num_spins, dropout_rate=dropout)# queries come from the previous layer of the decoder while the keys and values come from the encoder. Used in translation
        self.feed_forward = PositionWiseFeedForward(d_model=embed_dim, d_ff=proj_layer_dim)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output):
        attn_output = self.self_attn(x)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        x = x.sum(dim=0)
        return x

In [157]:
class Transformer(nn.Module):
    def __init__(self, embed_dim, a, max_seq_length, num_spins, proj_layer_dim, dropout):
        super(Transformer, self).__init__()
        #self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        #self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.word_embeddings = nn.Linear(num_spins, embed_dim)

        #self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        #self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.encoder_layer = EncoderLayer(embed_dim, a, max_seq_length, num_spins, proj_layer_dim, dropout)
        self.decoder_layer = DecoderLayer(embed_dim, a, max_seq_length, num_spins, proj_layer_dim, dropout)
        self.fc = nn.Linear(embed_dim, num_spins)

    def forward(self, src, tgt):
        #src_mask, tgt_mask = self.generate_mask(src, tgt)
        #src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        #tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))
        src_embedded = self.word_embeddings(src)
        tgt_embedded = self.word_embeddings(tgt)
        enc_output = self.encoder_layer(src_embedded)
        dec_output = self.decoder_layer(tgt_embedded, enc_output)
        output = self.fc(dec_output)
        return output

In [161]:
def evaluate(model, data_test, data_test_dec, vocab, criterion, device=0):
    model.eval()
    epoch_loss = 0
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    for i, data in tqdm.tqdm(enumerate(data_test), total=len(data_test)):
        # Get the inputs
        input_seq_enc = data
        input_seq_dec = data_test_dec[i]
        #input_one_hot = one_hot_encoding(input_seq.tolist(), vocab)
        input_decoder_one_hot = one_hot_encoding(input_seq_dec.tolist(), vocab)
        # mask a token
        masked_sequence_enc, masked_sequence_dec, position = mask_random_spin(input_seq_enc, input_seq_dec, vocab, mask_token=2)
        # Forward pass
        outputs = model.forward(masked_sequence_enc, masked_sequence_dec)

        #output_token = F.log_softmax(outputs, dim=-1)
        target_token = torch.argwhere(input_decoder_one_hot[position]==1).squeeze(0) #target_token = input_seq[position]

        # Compute loss
        loss = criterion(outputs.unsqueeze(0), target_token)
        epoch_loss += loss.item()
    #model.train()
    return epoch_loss / len(data_test)

def train(model, data_train, data_train_dec, data_test, data_test_dec, vocab, optimizer, criterion, num_epochs=8, device=0):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    # Training loop
    model.train()
    best_eval_loss = 1e-3 # used to do early stopping

    for epoch in tqdm.tqdm(range(num_epochs), leave=False, position=0):
        running_loss = 0
        epoch_loss = 0
        
        for i, data in tqdm.tqdm(enumerate(data_train), total=len(data_train)):
            # Get the inputs
            input_seq_enc = data
            input_seq_dec = data_train_dec[i]
            #input_encoder_one_hot = one_hot_encoding(input_seq_enc.tolist(), vocab)
            input_decoder_one_hot = one_hot_encoding(input_seq_dec.tolist(), vocab)

            # mask a token
            masked_sequence_enc, masked_sequence_dec, position = mask_random_spin(input_seq_enc, input_seq_dec, vocab, mask_token=2)

            # Forward pass
            prediction = model.forward(masked_sequence_enc, masked_sequence_dec) #masked_sequence[masked_position]
            
            #predicted_token = F.log_softmax(prediction, dim=-1)
            target_token = torch.argwhere(input_decoder_one_hot[position]==1).squeeze(0) #input_seq[masked_position]
            # Compute loss
            loss = criterion(prediction.unsqueeze(0), target_token)
            epoch_loss += loss.item()
            
            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 10 == 9. :    # print every 10 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0

        print(f'Epoch {epoch + 1} | Train Loss: {(epoch_loss / len(data)):.4f}')
        eval_loss = evaluate(model, data_test, data_test_dec, vocab, criterion, device=device)
        print(f'Epoch {epoch + 1} | Eval Loss: {(eval_loss):.4f}')
        
        # Perform early stopping based on eval loss
        if eval_loss < best_eval_loss:
            return epoch_loss / len(data_train)
    return epoch_loss / len(data_train)

In [162]:
# Define the parameters 
vocab_size = 3
vocab = {-1:0,1:1,2:2} 
L = 20
embedding_dim = 20
proj_layer_dim = 128
hidden_dim = 20
num_layers = 1 # have to adapt the model for 2 and 3 layers
dropout_rate = 0.0
lr = 1e-3
num_sequences = 1000
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [163]:
# Example usage:
model = Transformer(embed_dim=embedding_dim, a=0, max_seq_length=L, num_spins=3, proj_layer_dim=128, dropout=dropout_rate)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

train(model, data_train, data_train_dec, data_test, data_test_dec, vocab, optimizer, criterion, device=device)
#torch.save(model.state_dict(), 'models/lstm_scratch.pt')
#evaluate(model, test_dataloader, criterion, device=device)

  0%|          | 0/8 [00:00<?, ?it/s]/var/folders/mg/44g5ch495hdc8x4_5mq3dsdm0000gn/T/ipykernel_91438/1462526590.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(one_hot_1, dtype=torch.float), torch.tensor(one_hot_2, dtype=torch.float), torch.tensor(mask_position)


[1,    10] loss: 0.510
[1,    20] loss: 0.085
[1,    30] loss: 0.207
[1,    40] loss: 0.080
[1,    50] loss: 0.110
[1,    60] loss: 0.110
[1,    70] loss: 0.161
[1,    80] loss: 0.093
[1,    90] loss: 0.098
[1,   100] loss: 0.109


[1,   110] loss: 0.157
[1,   120] loss: 0.184
[1,   130] loss: 0.119
[1,   140] loss: 0.093
[1,   150] loss: 0.165
[1,   160] loss: 0.053
[1,   170] loss: 0.117
[1,   180] loss: 0.075
[1,   190] loss: 0.076


[1,   200] loss: 0.070
[1,   210] loss: 0.125
[1,   220] loss: 0.093
[1,   230] loss: 0.107
[1,   240] loss: 0.133
[1,   250] loss: 0.060


[1,   260] loss: 0.094
[1,   270] loss: 0.112
[1,   280] loss: 0.085
[1,   290] loss: 0.106
[1,   300] loss: 0.109
[1,   310] loss: 0.075
[1,   320] loss: 0.085
[1,   330] loss: 0.062
[1,   340] loss: 0.073


...

100%|██████████| 1000/1000 [00:02<00:00, 408.30it/s]


[8,   990] loss: 0.071
[8,  1000] loss: 0.052
Epoch 8 | Train Loss: 34.9304


100%|██████████| 400/400 [00:00<00:00, 1596.01it/s]
                                             

Epoch 8 | Eval Loss: 0.8282


0.6986083506941795

In [91]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(10, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1, 2, 4, 5], [4, 3, 2, 9]])
print(input.shape)
print(embedding(input).shape)


torch.Size([2, 4])
torch.Size([2, 4, 3])
